In [15]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import torch
from scipy.special import softmax

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification, pipeline

In [16]:
# Páginas web de noticias italianas, alemanas y francesas

url_IT="https://www.ilmessaggero.it/"

url_DE="https://www.focus.de/"  #Unico h3

url_FR="https://www.lexpress.fr/"

Titulares de la página italiana

In [17]:
page=requests.get(url_IT)

soup=bs(page.text,'html')

titulares_IT = soup.find_all('h2')
titulares_IT= [titulos.text  for titulos in titulares_IT]

titularesIT10 = titulares_IT[:10]
df= pd.DataFrame(data=titularesIT10, columns=["Titulares V.O."])

Titulares de la página alemana

In [18]:
page=requests.get(url_DE)

soup=bs(page.text,'html')

titulares_DE = soup.find_all('h3')
titulares_DE= [titulos.text  for titulos in titulares_DE]

titularesDE10 = titulares_DE[:10]
titularesDE10 = pd.Series(titularesDE10)
df = pd.concat([df, pd.DataFrame(titularesDE10, columns=["Titulares V.O."])], ignore_index=True)

Titulares de la página francesa

In [19]:
page=requests.get(url_FR)

soup=bs(page.text,'html')

titulares_FR = soup.find_all('h2')
titulares_FR= [titulos.text  for titulos in titulares_FR]

titularesFR10 = titulares_FR[:10]
titularesFR10 = pd.Series(titularesFR10)
df = pd.concat([df, pd.DataFrame(titularesFR10, columns=["Titulares V.O."])], ignore_index=True)

In [20]:
df

,Titulares V.O.
0,"LA STORIA Giulia, due lauree con lode ma nessu..."
1,"L'EVENTO Met Gala 2024, i look più belli sul r..."
2,"GUERRA Disfatta ucraina a Chasiv Yar, brigata ..."
3,"IL PERSONAGGIO Fabio Maria Damato, il manager..."
4,"Corruzione, Toti arrestato: «Siamo tranquillis..."
5,Dalle suite a Montecarlo alla borsa Chanel e l...
6,Briatore: «In Italia ti condannano subito. Con...
7,Kiev: «Due arresti per tentato omicidio Zelens...
8,Dalla possibile offensiva russa in estate alla...
9,Israele prende il valico di Rafah: «Proposta d...


# Traducciones:

In [21]:
model_nllb = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")
tokenizer_nllb = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")

Traduzco los titutlares en italiano

In [22]:
source_lang = "ita_Latn"
target_lang = "eng_Latn"
translator = pipeline("translation", model=model_nllb, tokenizer=tokenizer_nllb, src_lang=source_lang, tgt_lang=target_lang, max_length = 400)

titularesIT10_traducidos = []

for i in titularesIT10:
    output = translator(i)
    translated_text = output[0]["translation_text"]
    titularesIT10_traducidos.append(translated_text)

Traduzco los titutlares en alemán

In [23]:
source_lang = "deu_Latn"
target_lang = "eng_Latn"
translator = pipeline("translation", model=model_nllb, tokenizer=tokenizer_nllb, src_lang=source_lang, tgt_lang=target_lang, max_length = 400)

titularesDE10_traducidos = []

for i in titularesDE10:
    output = translator(i)
    translated_text = output[0]["translation_text"]
    titularesDE10_traducidos.append(translated_text)

Traduzco los titutlares en francés

In [24]:
source_lang = "fr"
target_lang = "eng_Latn"
translator = pipeline("translation", model=model_nllb, tokenizer=tokenizer_nllb, src_lang=source_lang, tgt_lang=target_lang, max_length = 400)

titularesFR10_traducidos = []

for i in titularesFR10:
    output = translator(i)
    translated_text = output[0]["translation_text"]
    titularesFR10_traducidos.append(translated_text)

Añado las traducciones al dataset

In [25]:
titulares_traducidos = titularesIT10_traducidos + titularesDE10_traducidos + titularesFR10_traducidos
df_traducciones = pd.DataFrame({"Titulares traducidos":titulares_traducidos})

df = pd.concat([df,df_traducciones], axis=1)
df

,Titulares V.O.,Titulares traducidos
0,"LA STORIA Giulia, due lauree con lode ma nessu...","The story of Julia, two graduates with praise ..."
1,"L'EVENTO Met Gala 2024, i look più belli sul r...","The event Met Gala 2024, the most beautiful lo..."
2,"GUERRA Disfatta ucraina a Chasiv Yar, brigata ...","The Ukrainian defeat at Chasiv Yar, brigade ""d..."
3,"IL PERSONAGGIO Fabio Maria Damato, il manager...","Fabio Maria Damato, the manager of Ferragni br..."
4,"Corruzione, Toti arrestato: «Siamo tranquillis...","Corruption, arrested: We are very calm Prosecu..."
5,Dalle suite a Montecarlo alla borsa Chanel e l...,From suites in Monte Carlo to the Chanel stock...
6,Briatore: «In Italia ti condannano subito. Con...,"I've known John for 30 years, I'm convinced of..."
7,Kiev: «Due arresti per tentato omicidio Zelens...,Kiev: Two arrests for attempted murder Zelensk...
8,Dalla possibile offensiva russa in estate alla...,From the possible Russian offensive in the sum...
9,Israele prende il valico di Rafah: «Proposta d...,Israel takes the Rafah line: Proposal for an u...


# Análisis de sentimientos

In [26]:
tokenizer = AutoTokenizer.from_pretrained("syedkhalid076/RoBERTa-Sentimental-Analysis-Model")
model = AutoModelForSequenceClassification.from_pretrained("syedkhalid076/RoBERTa-Sentimental-Analysis-Model")

In [27]:
sentimientos = []
num_sentimientos_pos = 0
num_sentimientos_neg = 0

for i in titulares_traducidos:
    inputSentiment = tokenizer(i, return_tensors="pt")

    outputSentiment = model(**inputSentiment)
    scoresSentiment = outputSentiment[0][0].detach().numpy()
    scoresSentiment = softmax(scoresSentiment)

    positive_score = scoresSentiment[1]
    negative_score = scoresSentiment[0]

    if positive_score > negative_score:
        sentimientos.append("Positive")
        num_sentimientos_pos = num_sentimientos_pos + 1
    else:
        sentimientos.append("Negative")
        num_sentimientos_neg = num_sentimientos_neg + 1
    
# Añado los sentimientos al dataframe
df["Sentimientos de las frases"] = sentimientos
df

,Titulares V.O.,Titulares traducidos,Sentimientos de las frases
0,"LA STORIA Giulia, due lauree con lode ma nessu...","The story of Julia, two graduates with praise ...",Negative
1,"L'EVENTO Met Gala 2024, i look più belli sul r...","The event Met Gala 2024, the most beautiful lo...",Positive
2,"GUERRA Disfatta ucraina a Chasiv Yar, brigata ...","The Ukrainian defeat at Chasiv Yar, brigade ""d...",Negative
3,"IL PERSONAGGIO Fabio Maria Damato, il manager...","Fabio Maria Damato, the manager of Ferragni br...",Positive
4,"Corruzione, Toti arrestato: «Siamo tranquillis...","Corruption, arrested: We are very calm Prosecu...",Positive
5,Dalle suite a Montecarlo alla borsa Chanel e l...,From suites in Monte Carlo to the Chanel stock...,Negative
6,Briatore: «In Italia ti condannano subito. Con...,"I've known John for 30 years, I'm convinced of...",Positive
7,Kiev: «Due arresti per tentato omicidio Zelens...,Kiev: Two arrests for attempted murder Zelensk...,Negative
8,Dalla possibile offensiva russa in estate alla...,From the possible Russian offensive in the sum...,Negative
9,Israele prende il valico di Rafah: «Proposta d...,Israel takes the Rafah line: Proposal for an u...,Negative


Resumen de las noticias

In [28]:
print(f"Podemos analizar rápidamente los sentiminetos de las noticias de diferentes países.\nEn este caso, tenemos {num_sentimientos_pos} noticias positivas y {num_sentimientos_neg}, noticias negativas")

Podemos analizar rápidamente los sentiminetos de las noticias de diferentes países.
En este caso, tenemos 12 noticias positivas y 13, noticias negativas
